In [159]:
import csv
import json
from copy import deepcopy
import pickle as pkl
csv.field_size_limit(1131072)

1131072

In [160]:
from annotation.analysis.utils.data_util import generate_all_clusters_combine_speakers
from annotation.analysis.utils.data_util import generate_clusters_no_plural_combine_speakers
from annotation.analysis.utils.data_util import collect_mentions
from annotation.adjudication.utils import read_turkle_annotation_multiple_scene, gather_by_scene
from annotation.adjudication.utils import extract_common_cluster, add_to_common_cluster
from annotation.adjudication.utils import analyze_types_of_difference, get_disagreement_types, get_error_matrix

In [161]:
data = []
temp = read_turkle_annotation_multiple_scene('source_data/institute_name_5_17.csv')
data.extend(temp)
temp = read_turkle_annotation_multiple_scene('source_data/institute_name_5_20.csv')
data.extend(temp)
temp = read_turkle_annotation_multiple_scene('source_data/institute_name_6_7.csv')
data.extend(temp)

In [162]:
temp = data[0]
for item in temp:
    print(item)
    print(temp[item])
    print()

sentences
[['Leonard', ':', 'Sheldon', ',', 'it', "'s", 'not', 'a', 'great', 'time', '!'], ['Sheldon', ':', 'I', "'m", 'sorry', ',', 'but', 'this', 'is', 'important', '.'], ['Leonard', ':', 'What', 'is', 'it', '?'], ['Sheldon', ':', 'Back', 'to', 'the', 'Future', 'II', 'was', 'in', 'the', 'Back', 'to', 'the', 'Future', 'III', 'case', '...', 'and', 'Back', 'to', 'the', 'Future', 'III', 'was--', 'get', 'this--', 'in', 'the', 'Back', 'to', 'the', 'Future', 'II', 'case', '.'], ['Leonard', ':', 'So', '?'], ['Sheldon', ':', 'So', ',', 'did', 'you', 'do', 'that', ',', 'or', 'am', 'I', 'in', 'the', 'house', 'with', 'an', 'intruder', '?'], ['Leonard', ':', 'Sheldon', ',', 'I', 'got', 'to', 'go', 'inside', '.', 'It', "'s", 'getting', 'rough', 'out', 'here', '.'], ['Sheldon', ':', 'You', "'re", 'dodging', 'the', 'question', ';', 'I', 'knew', 'it', 'was', 'you', '.'], ['Leonard', ':', 'What', 'was', 'what', '?'], ['Sheldon', ':', 'This', 'is', "n't", 'a', 'very', 'good', 'connection', ',', 'but', 

In [163]:
# Load Data and relabel the speaker token
data = []
temp = read_turkle_annotation_multiple_scene('source_data/institute_name_5_17.csv')
data.extend(temp)
temp = read_turkle_annotation_multiple_scene('source_data/institute_name_5_20.csv')
data.extend(temp)
temp = read_turkle_annotation_multiple_scene('source_data/institute_name_6_7.csv')
data.extend(temp)

# Transform speaker token to string
for item in data:
    temp = []
    for i, (query, answers) in enumerate(item['answers']):
        if isinstance(answers, list):
            for j, answer in enumerate(answers):
                if answer[1]==0 and answer[2]==1 and answer[0]<=(len(item['sentences'])-1):
                    item['answers'][i][1][j] = item['sentences'][answer[0]][0]
                else:
                    item['answers'][i][1][j] = "_".join([str(token) for token in answer])
        else:
            item['answers'][i][1] = [item['answers'][i][1]]
        item['answers'][i][1] = sorted(item['answers'][i][1])

results = []
results.extend(data)
scenes = gather_by_scene(results)

one_way = {}
two_way = {}
for item in scenes:
    if len(scenes[item])==1:
        one_way[item] = scenes[item]
    else:
        two_way[item] = scenes[item]
print(len(two_way), len(one_way))

268 16


In [164]:
def convert_annotation_to_query(annotations, sentences):
    query_option = {
            "mentions": [],
            "others": []
        }
    for token in annotations:
            token = token.strip().split("_")
            if len(token)==1:
                query_option['others'].append(token[0])
            elif len(token)==3:
                query_option['mentions'].append({
                    'sentenceIndex': token[0],
                    'startToken': token[1],
                    'endToken': token[2]
                })
    for token in annotations:
        token = token.strip().split("_")
        if len(token)==3:
            sent_idx, start, end  = [int(item) for item in token]
            try:
                text = " ".join(sentences[sent_idx][start: end])
            except:
                text = " "
            query_option['others'].append(text)
            print(text)

    return query_option

In [165]:
# Count Error Matrix
output = []
matrix_all = [[] for i in range(7)]
for scene_id in two_way:
    # if scene_id!="s07e17c07t":
    #     continue
    scene = two_way[scene_id]
    answers1, answers2 = scene[0]['answers'], scene[1]['answers']
    labels1, labels2 = get_disagreement_types(answers1, answers2)
    sentences = scene[0]['sentences']
    candidate_spans = scene[0]['query_spans']

    temp = {
        "sentences": sentences,
        "candidateSpans": candidate_spans,
        "clickSpans": candidate_spans,
        "scene_ids": [scene_id]
    }

    query_spans = []
    query_option1 = []
    query_option2 = []
    for a1, a2, l1, l2 in zip(answers1, answers2, labels1, labels2):
        if l1==l2:
            continue
        query_token = a1[0]
        # Add to query spans
        query_spans.append({
            'sentenceIndex': query_token[0],
            'startToken': query_token[1],
            'endToken': query_token[2],
        })
        query_option1.append(convert_annotation_to_query(a1[1], sentences))
        query_option2.append(convert_annotation_to_query(a2[1], sentences))

    temp['querySpans'] = query_spans
    temp['option1'] = query_option1
    temp['option2'] = query_option2

    output.append(temp)

this
This
kraken
What
you
you
Leonard
Leonard
Leonard
this oil stain
this oil stain
Lucy
Lucy
Lucy
welcome party
a welcome party
someone
this oil stain
your wife
The hotel
There
There
the bar
in the bed
Howie
Howie
Howie
Howie
Leonard
You
your work
jellyfish neural nets
my paper
it
it
undergrad
her
Mrs Davis
her
Mrs Davis
her
Mrs Davis
Mrs. Davis
Mrs Davis
Mrs. Davis
Mrs Davis
posterior cingulate cortex lesions
they
two gentlemen
the two gentlemen
two gentlemen
the two gentlemen
Boys
Paul Simon
him
Penny
it
ladies
Everybody
ladies
5:30 in the morning
5:30 in the morning
the morning
it
All
everybody
All
everybody
him
Leonard
them
them
them
them
no one
this
That
that
that
He
that guy
he
that
that
penguins
penguins
99 %
heartbreak
Here
beverage
beverage
idea
that
how many underpants
topless scene
a topless scene
movie
a low - budget horror movie
your big revelation
it
something
shared something
it
shared something
it
that
killer gorilla
It
strangers
strangers
one
one
the short one
a man
m

In [166]:
with open('adjudication_input/adjudication.pkl', 'wb') as f:
    pkl.dump(output, f)

In [167]:
print(len(output))

268
